In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
%matplotlib inline

In [40]:
combined_df = pd.read_csv('combined_df.csv', index_col='date', infer_datetime_format=True, parse_dates=True)

In [41]:
X = combined_df.drop(columns = 'class')
y = combined_df['class'].to_frame()
# standardize X values
X = (X - X.mean())/X.std()

In [42]:
# define functions to change dataframe to array
def target(df, column, steps):
    target = []
    for i in range(len(df) - steps):
        t = df.iloc[i + steps, column]
        target.append(t)
    return np.array(target).reshape(-1, 1)

def features(df, columns, steps):
    features = []
    for c in range(len(columns)):
        feature = []
        for i in range(len(df) - steps):
            f = list(df.iloc[i : i + steps, columns[c]])
            feature.append(f)
        features.append(feature)
    return np.array(features).T

In [43]:
X = features(X, list(range(8)), 10)
y = target(y, 0, 10)

In [16]:
X.shape

(10, 1440, 8)

In [44]:
y.shape

(1440, 1)

In [17]:
X = X.reshape(1440, 10, 8)

In [18]:
tt_split = int(len(X) * 0.8)
X_train = X[: tt_split]
X_test = X[tt_split :]
y_train = y[: tt_split]
y_test = y[tt_split :]

In [19]:
# one-hot-encode y values
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [25]:
model = Sequential()
neurons = 30
model.add(LSTM(units=neurons, return_sequences=True, input_shape=(X.shape[1], 8)))
model.add(Dropout(0.2))
model.add(LSTM(units=neurons))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['categorical_accuracy'])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 30)            4680      
                                                                 
 dropout_2 (Dropout)         (None, 10, 30)            0         
                                                                 
 lstm_3 (LSTM)               (None, 30)                7320      
                                                                 
 dropout_3 (Dropout)         (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 93        
                                                                 
Total params: 12,093
Trainable params: 12,093
Non-trainable params: 0
_________________________________________________________________


In [28]:
# define earliystopping
stop = EarlyStopping(monitor='categorical_accuracy', mode='min', patience=10, verbose=1)

In [29]:
model.fit(X_train, y_train, epochs=100, batch_size=5, verbose=1, shuffle=False, callbacks=[stop])

Epoch 1/100
231/231 [==============================] - 3s 4ms/step - loss: 0.3050 - categorical_accuracy: 0.9332
Epoch 2/100
231/231 [==============================] - 1s 4ms/step - loss: 0.2137 - categorical_accuracy: 0.9575
Epoch 3/100
231/231 [==============================] - 1s 4ms/step - loss: 0.2161 - categorical_accuracy: 0.9575
Epoch 4/100
231/231 [==============================] - 1s 4ms/step - loss: 0.2147 - categorical_accuracy: 0.9575
Epoch 5/100
231/231 [==============================] - 1s 4ms/step - loss: 0.2079 - categorical_accuracy: 0.9575
Epoch 6/100
231/231 [==============================] - 1s 5ms/step - loss: 0.2013 - categorical_accuracy: 0.9575
Epoch 7/100
231/231 [==============================] - 1s 4ms/step - loss: 0.2041 - categorical_accuracy: 0.9575
Epoch 8/100
231/231 [==============================] - 1s 4ms/step - loss: 0.1989 - categorical_accuracy: 0.9575
Epoch 9/100
231/231 [==============================] - 1s 4ms/step - loss: 0.1997 - categorical_

In [31]:
score = model.evaluate(X_test, y_test)[1]
print("%0.2f accuracy" % (score))

9/9 [==============================] - 0s 2ms/step - loss: 0.1799 - categorical_accuracy: 0.9653
0.97 accuracy


In [36]:
predicted = model.predict(X_test)
predicted

array([[9.82827067e-01, 1.50605096e-02, 2.11240957e-03],
       [9.81714368e-01, 3.82085145e-03, 1.44646699e-02],
       [9.84085798e-01, 3.91452340e-03, 1.19997310e-02],
       [9.93256927e-01, 5.49637899e-03, 1.24670332e-03],
       [9.94063199e-01, 2.16234475e-03, 3.77455680e-03],
       [9.95687187e-01, 2.99903401e-03, 1.31367228e-03],
       [9.92238224e-01, 6.61931932e-03, 1.14246435e-03],
       [9.90383685e-01, 8.37187283e-03, 1.24449295e-03],
       [9.94147062e-01, 3.62479454e-03, 2.22817296e-03],
       [9.82366920e-01, 2.32587755e-03, 1.53072439e-02],
       [9.96826291e-01, 1.50482752e-03, 1.66888186e-03],
       [9.96985137e-01, 1.54787104e-03, 1.46693364e-03],
       [9.94583189e-01, 2.52989028e-03, 2.88695865e-03],
       [9.89309430e-01, 9.31565929e-03, 1.37492165e-03],
       [9.92918074e-01, 2.13091169e-03, 4.95094480e-03],
       [9.69946802e-01, 2.49687377e-02, 5.08442381e-03],
       [9.75791037e-01, 4.39502485e-03, 1.98138971e-02],
       [9.13995743e-01, 2.83633

In [37]:
result = np.where(predicted>0.333, 1, 0)

In [38]:
result

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1,